We want to create a script that allows the end user to add data, in bulk, to an
existing web feature layer in an ArcGIS Enterprise portal.

The input is a csv file of X,Y data: "C:\Users\tim10393\Desktop\Crime_XY.csv"
Let's start by creating a script that allows the end user to append that Crime_XY csv file
to the Crime Incidents layer.

Let's first create a python script. Then, a python script tool. Then, a Python Tool box (all-in-one file)
Be thinking - what steps should be take to do this?
Step 1 - declare variables and import packages

In [1]:
import arcpy
import csv
import arcgis
from arcgis.gis import GIS

Things to change
- CSV File
- URL for the web layer
- Account for column names with X Y, to avoid using JUST an index position.
- Account for columns for the taget feature service
- Coordinate system for feature service 
- Coordinate system for the input coordiantes (csv file)


In [4]:
# Step 2, get access to the web layer in the portal.
gis = GIS("Pro", verify_cert=False)
print("Signed in!")

Setting `verify_cert` to False is a security risk, use at your own risk.


Signed in!


In [5]:
item = gis.content.get("140bb70ddb104eb196489e3fbc53648f")
crimeIncidents = item.layers[0]
print(crimeIncidents.url)

AttributeError: 'NoneType' object has no attribute 'layers'

In [ ]:
arcpy.env.overwriteOutput=True
crimeLyr = arcpy.management.MakeFeatureLayer(
    in_features=crimeIncidents.url,
    out_layer="CrimePoints",
    where_clause="",
    workspace=None,
    field_info="OBJECTID OBJECTID VISIBLE NONE;IncidentName IncidentName VISIBLE NONE;Time_stamp Time_stamp VISIBLE NONE;Shape Shape VISIBLE NONE"
)
crimeLyr[0]

In [ ]:
fieldNames = ['IncidentName','Time_stamp',"Shape@"]
iCur = arcpy.da.InsertCursor(crimeLyr[0],fieldNames)

In [ ]:
csvFile = r"C:\Users\tim10393\Desktop\Crime_XY.csv"
file = open(csvFile)
csvData = csv.reader(file)
next(csvData)

GCS_WGS1984_sp = arcpy.SpatialReference(4326)
PCS_WMAS_sp = arcpy.SpatialReference(3857)

for row in csvData:
    print(row)
    pnt = arcpy.Point(row[2],row[3])
    pntGeom = arcpy.PointGeometry(pnt, spatial_reference=GCS_WGS1984_sp)
    pntGeom = pntGeom.projectAs(PCS_WMAS_sp)
    iName = None
    timeStamp = row[1]
    iRow = [iName, timeStamp, pntGeom]
    iCur.insertRow(iRow)
del iCur, csvData